This notebook does correlation between your Kandji, Intune, Crowdstrike, and AssetPanda devices.\
Originally written to find devices in MDM but not Crowdstrike.\
The code requires you to download CSVs of: your Kandji, Intune, Crowdstrike, and AssetPanda devices.

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
intune = pd.read_csv('./data/intune.csv')
kandji = pd.read_csv('./data/kandji.csv')
cshosts = pd.read_csv('./data/crowdstrike.csv')
assetpanda = pd.read_csv('./data/assetpanda.csv')

timestamp = datetime.now().strftime("%Y-%m-%d")

In [ ]:
intune.shape

In [ ]:
kandji.shape

In [ ]:
cshosts.shape

In [ ]:
#create df for devices in intune but not cs
intune['Last check-in'] = pd.to_datetime(intune['Last check-in']).dt.tz_localize(None)

# Filter based on conditions: Serial not in cshosts['Serial Number'] and Checked In within the last 45 days
windows_not_in_cs = intune[
    (~intune['Serial number'].isin(cshosts['Serial Number'])) &
    (intune['Last check-in'] >= pd.to_datetime('today') - pd.DateOffset(days=45))
]
windows_not_in_cs.shape

In [ ]:
#create df for devices in kandji but not cs

# Convert 'Checked In' column to datetime if not already in datetime format
kandji['Checked In'] = pd.to_datetime(kandji['Checked In']).dt.tz_localize(None)

# Filter based on conditions: Serial not in cshosts['Serial Number'] and Checked In within the last 45 days
macs_not_in_cs = kandji[
    (~kandji['Serial'].isin(cshosts['Serial Number'])) &
    (kandji['Checked In'] >= pd.to_datetime('today') - pd.DateOffset(days=45)) &
    (kandji['Blueprint'] != 'IT Kiosk')
]
macs_not_in_cs.shape

In [ ]:
#dataframes to csv files
windows_not_in_cs.to_csv(f'results/windows-in-intune-but-not-cs-{timestamp}.csv', index=False)
macs_not_in_cs.to_csv(f'results/macs-in-kandji-but-not-cs-{timestamp}.csv', index=False)

In [ ]:
assetpanda_hosts_not_in_cs = assetpanda[assetpanda['Serial #'].isin(macs_not_in_cs['Serial']) | assetpanda['Serial #'].isin(windows_not_in_cs['Serial number'])]
assetpanda_hosts_not_in_cs.shape

In [ ]:
assetpanda_hosts_not_in_cs.to_csv(f'results/assetpanda-hosts-not-in-cs-{timestamp}.csv', index=False)

In [ ]:
cshosts_hosts_not_in_assetpanda = cshosts[~cshosts['Serial Number'].isin(assetpanda['Serial #'])]
cshosts_hosts_not_in_assetpanda.shape

In [ ]:
cshosts_hosts_not_in_assetpanda.to_csv(f'results/cshosts-not-in-assetpanda-{timestamp}.csv', index=False)

Hosts not in Crowdstike or Assetpanda

In [ ]:
windows_hosts_not_in_cs_or_assetpanda = windows_not_in_cs[~windows_not_in_cs['Serial number'].isin(assetpanda['Serial #'])]
windows_hosts_not_in_cs_or_assetpanda.shape

In [ ]:
mac_hosts_not_in_cs_or_assetpanda = macs_not_in_cs[~macs_not_in_cs['Serial'].isin(assetpanda['Serial #'])]
mac_hosts_not_in_cs_or_assetpanda.shape

In [ ]:
mdm_hosts_not_in_cs_or_assetpanda = pd.concat([mac_hosts_not_in_cs_or_assetpanda, windows_hosts_not_in_cs_or_assetpanda], ignore_index=True)
mdm_hosts_not_in_cs_or_assetpanda.shape

In [ ]:
mdm_hosts_not_in_cs_or_assetpanda.to_csv(f'results/45-day-mdm-hosts-not-in-cs-or-assetpanda-{timestamp}.csv', index=False)

Crowdstike hosts not in MDM

In [ ]:
cshosts_not_in_intune = cshosts[(cshosts['Platform'] == 'Windows') & (~cshosts['Serial Number'].isin(intune['Serial number']))]
print(cshosts_not_in_intune.shape)

In [ ]:
cshosts_not_in_kandji = cshosts[(cshosts['Platform'] == 'Mac') & (~cshosts['Serial Number'].isin(kandji['Serial']))]
print(cshosts_not_in_kandji.shape)

In [ ]:
# adding assetpanda to macs-in-cs-but-not-kandji
cshosts_not_in_kandji = pd.merge(cshosts_not_in_kandji, assetpanda, left_on='Serial Number', right_on='Serial #', how='left')

In [ ]:
#dataframes to csv files
cshosts_not_in_intune.to_csv(f'results/windows-in-cs-but-not-intune-{timestamp}.csv', index=False)
cshosts_not_in_kandji.to_csv(f'results/macs-in-cs-but-not-kandji-{timestamp}.csv', index=False)